In [1]:
import pandas as pd
import numpy as np
import time
import os
import collections
from XFLib import *
# class time_est():
#     def __init__(self, total_len):
#         self.t_start = time.time()
#         self.total_len = total_len
#         self.count = 0
#         self.t_ref = time.time()
    
#     def check(self,no_of_check=1,info=""):
#         self.count += no_of_check
#         if time.time() - self.t_ref > 1:
#             t_used = time.time() - self.t_start
#             t_total = t_used * self.total_len / self.count
#             t_remain = t_total - t_used
#             process_bar = "|"
#             for i in range(40):
#                 if (i/40) < (self.count/self.total_len):
#                     process_bar += "█"
#                 else:
#                     process_bar += " "
#             process_bar += "|"
#             print("\r" + (str(info) + "  {:.2f}% ({}/{})\t".format(self.count * 100/self.total_len, self.count,self.total_len)) + str(process_bar).ljust(45) + "Used: {:02.0f}:{:02.0f}:{:02.0f}".format(int(t_used/3600), int(t_used/60)%60, t_used % 60).ljust(16) + "ETA: {:02.0f}:{:02.0f}:{:02.0f}".format(int(t_remain/3600), int(t_remain/60)%60, t_remain % 60),end="")
#             self.t_ref = time.time()
#         if self.count == self.total_len:
#             t_used = time.time() - self.t_start
#             print("\r" + str(info) + "  Finished in " +"{:02.0f}:{:02.0f}:{:02.0f}".format(int(t_used/3600), int(t_used/60)%60, t_used % 60).ljust(100))

In [2]:
path = 'i2v_sep/'
file = os.listdir(path)

In [5]:
top_artist = np.load('eval/artist.npy')

In [7]:
customer_id_dict = collections.defaultdict()
est = time_est(len(file))
for i in file:
    df = pd.read_csv(path+i, low_memory = False)
    df = df.drop('Unnamed: 0',1)
    df = df[df['day']!='day']
    df['hour'] = [int(x) for x in df['hour']]
    df['minute'] = [int(x) for x in df['minute']]
    df = df.sort_values(['hour','minute'])
    df = df[df['artist_name'].isin(top_artist)]
    df_np = np.array(df)
    for customer_id, artist_name, _, _, _ in df_np:
        try:
            customer_id_dict[customer_id].append(artist_name)
        except:
            customer_id_dict[customer_id] = []
            customer_id_dict[customer_id].append(artist_name)
    est.check()

Finished in 00:27:51                                                                                            


In [45]:
len(df['artist_name'].value_counts())

99

In [9]:
np.save('eval/i2v_cus.npy', customer_id_dict)

In [11]:
artist_info = list(customer_id_dict.values())

In [12]:
len(artist_info)

3961520

In [16]:
import gensim

In [17]:
model = gensim.models.Word2Vec(artist_info, size=100, window=5, min_count=5, workers=4)

In [22]:
model.save('i2v/i2v_100')

In [20]:
model.most_similar('Linkin Park')

[('Disturbed', 0.3888905942440033),
 ('Of Mice & Men', 0.3714606761932373),
 ('Skillet', 0.3460681736469269),
 ('Asking Alexandria', 0.34110310673713684),
 ('Enya', 0.3410044014453888),
 ('Iron Maiden', 0.3347916603088379),
 ('Avenged Sevenfold', 0.3299633860588074),
 ('The Streets', 0.32304951548576355),
 ('Pendulum', 0.320709228515625),
 ('Bring Me The Horizon', 0.3195466995239258)]

In [24]:
genres = np.load('eval/top100artistgenres.npy').item()

In [37]:
ip = 'Linkin Park'

In [41]:
related = list(model.most_similar(ip))

In [42]:
related = [x[0] for x in related]

In [43]:
related

['Disturbed',
 'Of Mice & Men',
 'Skillet',
 'Asking Alexandria',
 'Enya',
 'Iron Maiden',
 'Avenged Sevenfold',
 'The Streets',
 'Pendulum',
 'Bring Me The Horizon']

In [44]:
ip_gen = genres[ip]

count = 0
for j in related:
    for i in genres[j]:
        if i in ip_gen:
            count+=1
            break
print(count/10)

0.8


In [46]:
count = 0
for i in top_artist:
    related = list(model.most_similar(i))
    related = [x[0] for x in related]
    ip_gen = genres[ip]
    for j in related:
        for i in genres[j]:
            if i in ip_gen:
                count+=1
                break

In [52]:
count/1000

0.705

In [53]:
len(top_artist)

100